In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
movieData = spark.read.load("dbfs:/FileStore/tables/movie_metadata.tsv", format="csv", sep="\t",
inferSchema="true", header="false")

In [0]:
movieTable = movieData.select('_c0','_c2').toDF('movieID', 'movieName')

In [0]:
plots = sc.textFile("/FileStore/tables/plot_summaries.txt")

In [0]:
N = plots.count()

In [0]:
plots = plots.map(lambda x: x.split('\t')).map(lambda x: (x[0],x[1].replace(',','').replace('.','').replace('?','').replace('\"','').lower().split(' ')))

In [0]:
plotsDF = plots.toDF(["movieId","RawSummary"])

In [0]:
from pyspark.ml.feature import StopWordsRemover
cleanData = StopWordsRemover(inputCol='RawSummary',outputCol='CleanSummary')

In [0]:
# display(cleanData.transform(plotsDF))

In [0]:
plotsDF = cleanData.transform(plotsDF).select('movieId','CleanSummary')

In [0]:
# display(plotsDF)

In [0]:
from pyspark.sql.functions import explode
plotsRDD = plotsDF.select(plotsDF.movieId, explode(plotsDF.CleanSummary)).rdd

In [0]:
plot_wc = plotsRDD.map(lambda x: ((x[0], x[1]),1)).reduceByKey(lambda x,y: x+y)

In [0]:
# plot_wc.collect()

In [0]:
word_tf = plot_wc.map(lambda x: (x[0][1],(x[0][0],x[1])))

In [0]:
n = plot_wc.map(lambda x: (x[0][1], 1)).reduceByKey(lambda x, y: x+y)

In [0]:
# word_tf.take(10)

In [0]:
# n.take(10)

In [0]:
# word_tf.count()

In [0]:
from math import log
param_w = n.map(lambda x: (x[0], log(N/x[1])))

In [0]:
# param_w.take(5)

In [0]:
w = word_tf.join(param_w)

In [0]:
# w.take(10)

In [0]:
final_weights = w.map(lambda x: (x[0], (x[1][0][0], x[1][0][1], x[1][1],(x[1][0][1]*x[1][1]))))

In [0]:
final_weights.take(10)

Out[49]: [('style', ('17060199', 1, 4.3052949892910855, 4.3052949892910855)),
 ('style', ('4057855', 1, 4.3052949892910855, 4.3052949892910855)),
 ('style', ('11470344', 1, 4.3052949892910855, 4.3052949892910855)),
 ('style', ('5080406', 2, 4.3052949892910855, 8.610589978582171)),
 ('style', ('468353', 1, 4.3052949892910855, 4.3052949892910855)),
 ('style', ('16673963', 1, 4.3052949892910855, 4.3052949892910855)),
 ('style', ('6563913', 1, 4.3052949892910855, 4.3052949892910855)),
 ('style', ('12561843', 1, 4.3052949892910855, 4.3052949892910855)),
 ('style', ('25185963', 1, 4.3052949892910855, 4.3052949892910855)),
 ('style', ('19024755', 1, 4.3052949892910855, 4.3052949892910855))]

In [0]:
single_term_search = sc.textFile("/FileStore/tables/single_term_search.txt")

In [0]:
terms = single_term_search.collect()

In [0]:
for term in terms:
    print(term + ":")
    topMovies = final_weights.filter(lambda x: x[0] == term).sortBy(lambda x: -x[1][3]).map(lambda x: (x[1][0], x[1][3])).take(10)
    tM = sc.parallelize(topMovies)
    topMovieTable = tM.toDF(['movieID', 'tf-idf_Weights'])
    result = movieTable.join (topMovieTable,movieTable.movieID == topMovieTable.movieID, "inner")
    result.select('movieName', 'tf-idf_Weights').show(truncate = False)

comedy:
+------------------------------------+------------------+
|movieName                           |tf-idf_Weights    |
+------------------------------------+------------------+
|Man on the Moon                     |17.51168871082063 |
|General Motors 50th Anniversary Show|17.51168871082063 |
|Talkin' Dirty After Dark            |17.51168871082063 |
|Where the Truth Lies                |17.51168871082063 |
|Hollywood Outlaw Movie              |17.51168871082063 |
|Cinta Kura Kura                     |13.133766533115471|
|Mahaul Theek Hai                    |13.133766533115471|
|Thousands Cheer                     |13.133766533115471|
|Funny Bones                         |13.133766533115471|
|When Stand Up Stood Out             |13.133766533115471|
+------------------------------------+------------------+

romance:
+------------------------------+------------------+
|movieName                     |tf-idf_Weights    |
+------------------------------+------------------+
|4 Romance    

In [0]:
query = sc.textFile("/FileStore/tables/multi_term.txt")
query_word = query.flatMap(lambda x: x.split(' ')).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)

In [0]:
query_word.collect()

Out[119]: [('corleone', 2),
 ('michael', 2),
 ('superman', 1),
 ('hulk', 1),
 ('Elsa', 1),
 ('elizabeth', 1),
 ('batman', 1)]

In [0]:
tf_new = final_weights.map(lambda x :  (x[0], (x[1][0], x[1][3])))

In [0]:
joined_data = query_word.join(tf_new)
# joined_data.take(50)

Out[110]: [('superman', (1, ('18463637', 6.418577694349836))),
 ('superman', (1, ('14268823', 6.418577694349836))),
 ('superman', (1, ('30801363', 12.837155388699673))),
 ('superman', (1, ('1059701', 160.4644423587459))),
 ('superman', (1, ('6954737', 83.44151002654787))),
 ('superman', (1, ('8630242', 6.418577694349836))),
 ('superman', (1, ('362719', 57.767199249148526))),
 ('superman', (1, ('28001277', 102.69724310959738))),
 ('superman', (1, ('11252503', 51.34862155479869))),
 ('superman', (1, ('33240946', 44.930043860448855))),
 ('superman', (1, ('12428147', 12.837155388699673))),
 ('superman', (1, ('6550694', 6.418577694349836))),
 ('superman', (1, ('3358393', 6.418577694349836))),
 ('superman', (1, ('11252704', 32.092888471749184))),
 ('superman', (1, ('17605137', 6.418577694349836))),
 ('superman', (1, ('1972947', 6.418577694349836))),
 ('superman', (1, ('22248107', 6.418577694349836))),
 ('superman', (1, ('20433007', 64.18577694349837))),
 ('superman', (1, ('6178996', 64.18577

In [0]:
from math import sqrt
cosine_param = joined_data.map(lambda x : (x[1][1][0], (x[1][0] * x[1][1][1], x[1][0] **2, x[1][1][1] **2))).reduceByKey(lambda x,y : ((x[0] + y[0], x[1] + y[1], x[2] + y[2])))
cosine_sim = cosine_param.map(lambda x : (x[0], 1-x[1][0]/(sqrt(x[1][1]) * sqrt(x[1][2]))))
# cosine_sim.take(50)

Out[111]: [('14268823', 0.0),
 ('12428147', 0.0),
 ('3358393', 0.0),
 ('17605137', 0.0),
 ('1972947', 0.0),
 ('22248107', 0.0),
 ('11265627', 0.0),
 ('23625840', 0.0),
 ('11248834', 0.0),
 ('35784403', 0.0),
 ('1040965', 0.0),
 ('4729', 0.0),
 ('26340083', 0.0),
 ('17288991', 0.0),
 ('19025779', 0.0),
 ('3022675', 0.0),
 ('3150991', 0.0),
 ('23214099', 0.0),
 ('1412382', 0.025320204629606824),
 ('3021304', 0.0),
 ('17401665', 0.0),
 ('1096485', 0.0),
 ('2862798', 0.0),
 ('2372994', 0.0),
 ('2236329', 0.0),
 ('2399419', 0.0),
 ('8862866', 0.0),
 ('25129766', 0.0),
 ('2157292', 0.0),
 ('23808966', 0.0),
 ('73376', 0.0),
 ('684018', 0.0),
 ('16877039', 0.0),
 ('21241505', 0.0),
 ('11660374', 0.0),
 ('11383001', 0.0),
 ('254269', 0.0),
 ('31176974', 0.0),
 ('11195154', 0.0),
 ('1689394', 0.0),
 ('29454281', 0.0),
 ('2918524', 0.053047770657739024),
 ('782794', 0.0),
 ('8775743', 0.0),
 ('2300167', 0.0),
 ('28785017', 0.0),
 ('1246394', 0.0),
 ('31960682', 0.0),
 ('15682219', 0.0),
 ('10862

In [0]:
sorted_result = cosine_sim.sortBy(lambda x: -x[1])

In [0]:
query_DF = sorted_result.toDF(["movieID", "cosineSimilarity"])

In [0]:
query_resultDF = movieTable.join(query_DF,movieTable.movieID == query_DF.movieID, "inner")

In [0]:
query_resultDF.select('movieName', 'cosineSimilarity').show(10,False)
# display(query_resultDF.select('movieName', 'cosineSimilarity'))

+------------------------------------+--------------------+
|movieName                           |cosineSimilarity    |
+------------------------------------+--------------------+
|Disaster Movie                      |0.25945543528870707 |
|Obsession                           |0.249984308816523   |
|Justice League: Crisis on Two Earths|0.21539337753768262 |
|Standing Room Only                  |0.08973045413461567 |
|The Godfather Part III              |0.0886212308727008  |
|Heavy Traffic                       |0.0806183465201421  |
|The Godfather                       |0.05395477704849949 |
|Secret Admirer                      |0.053047770657739024|
|Superman/Batman: Apocalypse         |0.05000486545285454 |
|At Long Last Love                   |0.025320204629606824|
+------------------------------------+--------------------+
only showing top 10 rows

